In [1]:
%pip install -U keras-segmentation opencv-python imgaug progress

PATH_OF_DATA= './data'
IMAGES_SUFFIX='images'
GD_SUFFIX='groundtruth'
TRAIN_PREFIX='train'
VALIDATION_PREFIX='val'

Requirement already up-to-date: keras-segmentation in /opt/anaconda3/lib/python3.7/site-packages (0.2.0)
Requirement already up-to-date: opencv-python in /opt/anaconda3/lib/python3.7/site-packages (4.1.2.30)
Requirement already up-to-date: imgaug in /opt/anaconda3/lib/python3.7/site-packages (0.3.0)
Requirement already up-to-date: progress in /opt/anaconda3/lib/python3.7/site-packages (1.5)
Note: you may need to restart the kernel to use updated packages.


In [2]:
import imgaug as ia
import imgaug.augmenters as iaa
# TODO: add augmentations
seq = iaa.Sequential([
    iaa.Crop(px=(0, 16)), # crop images from each side by 0 to 16px (randomly chosen)
    iaa.Fliplr(0.5), # horizontally flip 50% of the images
    iaa.GaussianBlur(sigma=(0, 3.0)) # blur images with a sigma of 0 to 3.0
])

# source: https://divamgupta.com/image-segmentation/2019/06/06/deep-learning-semantic-segmentation-keras.html
def augment_seg( img , seg  ):
    aug_det = seq.to_deterministic() 
    image_aug = aug_det.augment_image( img )

    segmap = ia.SegmentationMapOnImage( seg , nb_classes=np.max(seg)+1 , shape=img.shape )
    segmap_aug = aug_det.augment_segmentation_maps( segmap )
    segmap_aug = segmap_aug.get_arr_int()

    return image_aug , segmap_aug

In [67]:
import os
from helpers import *

PREPPED_PREFIX = 'prepped'
def prepped_path(path, suffix=None):
    if suffix is None:
        path = prepped_path(path, GD_SUFFIX)
        path = prepped_path(path, IMAGES_SUFFIX)
        return path
    return path.replace(suffix, f'{PREPPED_PREFIX}_{suffix}')

# Loaded a set of images
TRAIN_IMG=0
VAL_IMG=1
TRAIN_GD=2
VAL_GD=3
SETS = [TRAIN_IMG, VAL_IMG, TRAIN_GD, VAL_GD]

PATHS = [None]*len(SETS)
PATHS[TRAIN_IMG] = f'{PATH_OF_DATA}/{TRAIN_PREFIX}_{IMAGES_SUFFIX}/'
PATHS[VAL_IMG] = f'{PATH_OF_DATA}/{VALIDATION_PREFIX}_{IMAGES_SUFFIX}/'
PATHS[TRAIN_GD] = f'{PATH_OF_DATA}/{TRAIN_PREFIX}_{GD_SUFFIX}/'
PATHS[VAL_GD] = f'{PATH_OF_DATA}/{VALIDATION_PREFIX}_{GD_SUFFIX}/'

# Clean .DS_store to avoid troubles listing images
for label in SETS:
    for path in [PATHS[label], prepped_path(PATHS[label])]:
        !rm -f {path}.DS_store

# Load images
imgs = [None]*len(SETS)
for label in SETS:
    files = os.listdir(PATHS[label])
    imgs[label] = [img_float_to_uint8(load_image(PATHS[label] + files[i])) for i in range(len(files))]
    
# Convert groundtruth between 0 and 1
GD = [TRAIN_GD, VAL_GD]
for label in GD:
    for i, img in enumerate(imgs[label]):
        #imgs[label][i] = img_float_to_uint8(img)
        imgs[label][i][img>0] = 1.
        
print(imgs[TRAIN_GD][0][imgs[TRAIN_GD][0]!=0])

[1 1 1 ... 1 1 1]


In [70]:
import cv2
from progress.bar import Bar

# Clean previous prepped images and...
# Write newly prepped images
for label in SETS:
    path=prepped_path(PATHS[label])
    !rm -f {path}*
    bar = Bar(f'Writing at {path}', max=len(imgs[label]))
    for i, img in enumerate(imgs[label]):
        cv2.imwrite(f'{path}img_{i}.png', cv2.cvtColor(img, cv2.COLOR_RGB2BGR))
        bar.next()
    bar.finish()

In [71]:
#print(imgs[TRAIN_IMG][0])
#print(img_float_to_uint8(mpimg.imread(prepped_path(PATHS[TRAIN_IMG])+'img_0.png')))
print(imgs[TRAIN_GD][0][imgs[TRAIN_GD][0]!=0])
print('--')
print(cv2.imread(prepped_path(PATHS[TRAIN_GD])+'img_0.png')[imgs[TRAIN_GD][0]!=0])

[1 1 1 ... 1 1 1]
--
[[1 1 1]
 [1 1 1]
 [1 1 1]
 ...
 [1 1 1]
 [1 1 1]
 [1 1 1]]
